# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import time
import pandas as pd
import requests

## Search for the TLDR Number `006179570C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [2]:
driver = webdriver.Chrome()
driver.get('https://www.tdlr.texas.gov/tools_search/')

In [3]:
tdlr_search = driver.find_element_by_id('mcrdata')
tdlr_search.send_keys('006179570C')

In [4]:
search_button = driver.find_element_by_id('submit3')
search_button.click()

In [30]:
driver.find_elements_by_tag_name('td')[5].text.replace('Name:   ','')

'BARRY MICHAEL SMITH'

In [44]:
driver.find_elements_by_tag_name('tr')[4].text

'Name:   BARRY MICHAEL SMITH DBA:   BUFFALO TOWING'

In [45]:
driver.find_elements_by_tag_name('td')[7].text.replace('Owner/Officer:   ','')

'BARRY MICHAEL SMITH / OWNER'

In [46]:
company_dict = {}
company_dict['business_name'] = driver.find_elements_by_tag_name('tr')[4].text #ugly, but necessary for below.
company_dict['owner'] = driver.find_elements_by_tag_name('td')[7].text.replace('Owner/Officer:   ','')
company_dict['phone'] = driver.find_elements_by_tag_name('td')[9].text.replace('Phone:   ','')
company_dict['status'] = driver.find_elements_by_tag_name('td')[12].text.replace('Status:  ','')
company_dict['address'] = driver.find_elements_by_tag_name('td')[14].text.split('\n')[-2:] #Cleaning this later

In [47]:
company_dict

{'business_name': 'Name:   BARRY MICHAEL SMITH DBA:   BUFFALO TOWING',
 'owner': 'BARRY MICHAEL SMITH / OWNER',
 'phone': '8066544404',
 'status': 'Active',
 'address': ['4501 W CEMETERY RD', 'CANYON, TX. 79015']}

# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [48]:
#Launching a driver:
driver = webdriver.Chrome()
driver.get('https://www.tdlr.texas.gov/tools_search/')
#Filling it out and clicking search:
tdlr_search = driver.find_element_by_id('mcrdata')
tdlr_search.send_keys('006179570C')
search_button = driver.find_element_by_id('submit3')
search_button.click()

company_dict = {}
company_dict['business_name'] = driver.find_elements_by_tag_name('tr')[4].text
company_dict['owner'] = driver.find_elements_by_tag_name('td')[7].text.replace('Owner/Officer:   ','')
company_dict['phone'] = driver.find_elements_by_tag_name('td')[9].text.replace('Phone:   ','')
company_dict['status'] = driver.find_elements_by_tag_name('td')[12].text.replace('Status:  ','')
company_dict['address'] = driver.find_elements_by_tag_name('td')[14].text.split('\n')[-2:] #Cleaning this later

print(company_dict)

{'business_name': 'Name:   B.D. SMITH TOWING DBA:   NO DATA', 'owner': 'BRANDT SMITH / OWNER', 'phone': '8173330706', 'status': 'Expired', 'address': ['13619 BRETT JACKSON RD.', 'FORT WORTH, TX. 76179']}


# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [49]:
df = pd.read_csv('trucks-subset.csv')
df.head()

,TDLR Number
0,006507931C
1,006179570C
2,006502097C


## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C`.

- *TIP: Use .apply and a function*
- *TIP: Unlike the Yelp example, you'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', -1)` to display aaaalll of the text in a cell*

In [50]:
# https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
def get_url(tldr_number):
    return 'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber='+tldr_number

In [51]:
pd.set_option('display.max_colwidth', -1)

df['TDLR Number'].apply(get_url)

0    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
Name: TDLR Number, dtype: object

### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [52]:
df['url'] = df['TDLR Number'].apply(get_url)

In [53]:
df

,TDLR Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Go through each row of the dataset, printing out information about each tow truck company.

Now will be **scraping** inside of your function.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

Just print it out for now.

- *TIP: use .apply*
- *TIP: You'll be using the code you wrote before, but converted into a function*
- *TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!*
- *TIP: Make sure you adjust any variables so you don't scrape the same page again and again*

In [54]:
#Launching a driver: driver = webdriver.Chrome()
def truck_scrape(url):
    driver.get(url)
    time.sleep(1)
    company_dict = {}
    company_dict['business_name'] = driver.find_elements_by_tag_name('tr')[4].text
    company_dict['owner'] = driver.find_elements_by_tag_name('td')[7].text.replace('Owner/Officer:   ','')
    company_dict['phone'] = driver.find_elements_by_tag_name('td')[9].text.replace('Phone:   ','')
    company_dict['status'] = driver.find_elements_by_tag_name('td')[12].text.replace('Status:  ','')
    company_dict['address'] = driver.find_elements_by_tag_name('td')[14].text.split('\n')[-2:]

    return company_dict

In [55]:
df.url.apply(truck_scrape)

0    {'business_name': 'Name:   AUGUSTUS E SMITH DBA:   A-1 AUTO RECOVERY', 'owner': 'AUGUSTUS EUGENE SMITH / OWNER', 'phone': '9032276464', 'status': 'Active', 'address': ['103 N MAIN ST', 'BONHAM, TX. 75418']}   
1    {'business_name': 'Name:   B.D. SMITH TOWING DBA:   NO DATA', 'owner': 'BRANDT SMITH / OWNER', 'phone': '8173330706', 'status': 'Expired', 'address': ['13619 BRETT JACKSON RD.', 'FORT WORTH, TX. 76179']}      
2    {'business_name': 'Name:   BARRY MICHAEL SMITH DBA:   BUFFALO TOWING', 'owner': 'BARRY MICHAEL SMITH / OWNER', 'phone': '8066544404', 'status': 'Active', 'address': ['4501 W CEMETERY RD', 'CANYON, TX. 79015']}
Name: url, dtype: object

## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

In [58]:
def truck_scrape(url):
    driver.get(url)
    time.sleep(1)
    return pd.Series({
        'business_name' : driver.find_elements_by_tag_name('tr')[4].text[5:],
        'owner' : driver.find_elements_by_tag_name('td')[7].text.replace('Owner/Officer:   ',''),
        'phone' : driver.find_elements_by_tag_name('td')[9].text.replace('Phone:   ',''),
        'status': driver.find_elements_by_tag_name('td')[12].text.replace('Status:  ',''),
        'address': driver.find_elements_by_tag_name('td')[14].text.split('\n')[-2:],
    })

In [61]:
df.url.apply(truck_scrape)

,business_name,owner,phone,status,address
0,AUGUSTUS E SMITH DBA: A-1 AUTO RECOVERY,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"[103 N MAIN ST, BONHAM, TX. 75418]"
1,B.D. SMITH TOWING DBA: NO DATA,BRANDT SMITH / OWNER,8173330706,Expired,"[13619 BRETT JACKSON RD., FORT WORTH, TX. 76179]"
2,BARRY MICHAEL SMITH DBA: BUFFALO TOWING,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"[4501 W CEMETERY RD, CANYON, TX. 79015]"


In [62]:
trucks_df = df.url.apply(truck_scrape).join(df)


In [64]:
trucks_df

,business_name,owner,phone,status,address,TDLR Number,url
0,AUGUSTUS E SMITH DBA: A-1 AUTO RECOVERY,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"[103 N MAIN ST, BONHAM, TX. 75418]",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,B.D. SMITH TOWING DBA: NO DATA,BRANDT SMITH / OWNER,8173330706,Expired,"[13619 BRETT JACKSON RD., FORT WORTH, TX. 76179]",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,BARRY MICHAEL SMITH DBA: BUFFALO TOWING,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"[4501 W CEMETERY RD, CANYON, TX. 79015]",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


### Save your dataframe as a CSV named `tow-trucks-extended.csv`

In [65]:
trucks_df.to_csv('tow-trucks-extended.csv', index=False)

### Re-open your dataframe to confirm you didn't save any extra weird columns

In [66]:
df_test = pd.read_csv('tow-trucks-extended.csv')
df_test

,business_name,owner,phone,status,address,TDLR Number,url
0,AUGUSTUS E SMITH DBA: A-1 AUTO RECOVERY,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"['103 N MAIN ST', 'BONHAM, TX. 75418']",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,B.D. SMITH TOWING DBA: NO DATA,BRANDT SMITH / OWNER,8173330706,Expired,"['13619 BRETT JACKSON RD.', 'FORT WORTH, TX. 76179']",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,BARRY MICHAEL SMITH DBA: BUFFALO TOWING,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"['4501 W CEMETERY RD', 'CANYON, TX. 79015']",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**

In [72]:
df = pd.read_csv('tow-trucks.csv')
df.shape

(20, 1)

In [73]:
df['url'] = df['TDLR Number'].apply(get_url)

In [75]:
tow_trucks_df = df.url.apply(truck_scrape).join(df)

In [76]:
tow_trucks_df

,business_name,owner,phone,status,address,TDLR Number,url
0,AUGUSTUS E SMITH DBA: A-1 AUTO RECOVERY,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"[103 N MAIN ST, BONHAM, TX. 75418]",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,B.D. SMITH TOWING DBA: NO DATA,BRANDT SMITH / OWNER,8173330706,Expired,"[13619 BRETT JACKSON RD., FORT WORTH, TX. 76179]",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,BARRY MICHAEL SMITH DBA: BUFFALO TOWING,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"[4501 W CEMETERY RD, CANYON, TX. 79015]",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,HEATH SMITH DBA: AUTOKRAFT COLLISION CENTER & WRECKER SRV,HEATH A SMITH / OWNER,940-552-0687,Expired,"[1529 WILBARGER ST, VERNON, TX. 76384]",006494912C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4,HEATH SMITH DBA: AUTO KRAFT COLLISION CENTER & WRECKER SV,HEATH A SMITH / OWNER,9405520687,Expired,"[1529 WILBARGER ST, VERNON, TX. 76384]",0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF
5,HYSMITH AUTOMOTIVE DBA: HYSMITH AUTOMOTIVE,WILLIAM THOMAS HYSMITH / PRESIDENT,Owner/Officer: ASHLEY ERIN HYSMITH / TREASURER,,[Status: Active],006448786C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006448786C
6,HYSMITH AUTOMOTIVE & TRUCK REPAIR INC DBA: HYSMITH AUTOMOTIVE,HYSMITH ERIN ASHLEY / TREASURER,Owner/Officer: WILLIAM THOMAS HYSMITH / PRESIDENT,,[Status: Suspended],0648444VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0648444VSF
7,HYSMITH AUTOMOTIVE & TRUCK REPAIR INC DBA: HYSMITH AUTOMOTIVE,WILLIAM THOMAS HYSMITH / PRESIDENT,Owner/Officer: ASHLEY ERIN HYSMITH / TREASURER,,[Status: Active],0651667VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0651667VSF
8,JEFF & WENDY SMITH DBA: A-1 WRECKER SERVICE,JEFF SMITH / PARTNER,Owner/Officer: WENDY SMITH / PARTNER,,[Status: Suspended],006017767C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006017767C
9,JEFF SMITH DBA: SAN JACINTO TOWING,JEFFREY JOHN SMITH / OWNER,8324354670,Active,"[4338 HARVEY RD, CROSBY, TX. 77532]",006495492C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C


In [79]:
#Yikes. My method of calling tr tags isn't very good when there are several owners + treasurers listed :-(
#I think the solution lies in including some if statements in the truck_scrape function. 